In [41]:
from requests import get
import json
import copy

In [42]:
def get_matches(nb_matches):
    response = get("http://api.localhost/matches")
    json_matches = json.loads(response.text)
    
    matches = []
    for match in json_matches:
        if (match['RI_A'] > 0):
            match['side'] = "A"
            matches.append(copy.deepcopy(match))
        if (match['RI_D'] > 0):
            match['side'] = "D"
            matches.append(copy.deepcopy(match))
        if (match['RI_H'] > 0):
            match['side'] = "H"
            matches.append(copy.deepcopy(match))

    return sorted(matches, key=lambda d: -d["RI_" + d["side"]])[0:nb_matches]

##### Bets 

Bet with an adaptable value for each match according to ratio E/V- with an amount of 1€.

In [43]:
def bet(amount, risk, matches):

    tot = 0
    for match in matches:
        side = match['side']
        match['esp'] = match['pred_' + side] * (match['odd_' + side] - 1) + (1 - match['pred_' + side]) * (-1);
        match['var_neg'] = (1 - match['pred_' + side]) * (-1 - match['esp']) ** 2;
        match['cred'] = match['esp']/match['var_neg']**(1-risk/5.0);
        tot += match['cred']
        
    for match in matches:
        match['cred'] = match['cred'] * amount / tot
    
    return matches

In [44]:
def evaluate_bets(matches):

    esp = 0
    var_neg = 0
    for match in matches:
        match['var_neg'] = match['var_neg'] * match['cred'] ** 2
        match['esp'] = match['esp'] * match['cred']
        esp += match['esp']
        var_neg +=  match['var_neg']
        
    ecart_type_neg = var_neg ** 0.5
    return [esp, ecart_type_neg]

##### Center the optimal ratio of gain/dispersion

* We define ratio as gain/dispersion because dispersion is related to the risk
    * We define gain as expected value E
    * We define risk as standart deviation or variance but only for negative values σ- / V-

Check values of ratio `E(X) / V-(X) ^ risk` in a range of `risk € [-5;5]` with a step of `0.1`.

_The value seems to be most of the time around of 1.0_ 

In [45]:
def center_optimal_risk(matches):
    opt = 0.0
    risk_opt = 0.0
    for risk in range(0,100):

        tot = 0
        for match in matches:
            side = match['side']
            match['esp'] = match['pred_' + side] * (match['odd_' + side] - 1) + (1 - match['pred_' + side]) * (-1);
            match['var_neg'] = (1 - match['pred_' + side]) * (-1 - match['esp']) ** 2;
            match['cred'] = match['esp']/match['var_neg']**((risk-50)/10.0);
            tot += match['cred']

        esp = 0
        var_neg = 0
        for match in matches:
            match['cred'] = match['cred'] / tot
            match['esp'] = match['esp'] * match['cred']
            match['var_neg'] = match['var_neg'] * match['cred'] ** 2
            esp += match['esp']
            var_neg +=  match['var_neg']
        
        ecart_type_neg = var_neg ** 0.5
        if opt < (esp / ecart_type_neg) :
            risk_opt = (risk - 50)/10.0
            opt = esp/ecart_type_neg
        #print(risk_opt, opt * 100 , "%")
    return risk_opt;

In [46]:
risk = 0
matches = get_matches(16)
print("opt risk",center_optimal_risk(matches))
amount = 100
matches = bet(amount, risk, matches)
[esp, ecart_type] = evaluate_bets(matches)
print(esp,ecart_type, "mise = ", amount, "risk = ", risk)
print("")
for match in matches:
    print(match['date'])
    print(match['match'] + " : " + match['side'], match['pred_' + match['side']])
    print(" bet =", match['cred'], "potentiel gain= ", match['cred'] * match['odd_' + match['side']])

('opt risk', 1.0)
(251.3692544487663, 60.10743800683271, 'mise = ', 100, 'risk = ', 0)

2020-3-1
(u'Milan AC - Genoa : A', 0.677936434745788)
(' bet =', 8.658742192646605, 'potentiel gain= ', 48.488956278820986)
2020-3-7
(u'Liverpool - Bournemouth : A', 0.5101109743118281)
(' bet =', 4.148759956026069, 'potentiel gain= ', 47.710739494299794)
2020-3-9
(u'Leicester - Aston Villa : A', 0.557868659496307)
(' bet =', 6.106721865697666, 'potentiel gain= ', 43.66306133973831)
2020-2-29
(u'Borussia Dortmund - Fribourg : A', 0.504249632358551)
(' bet =', 4.137456361632178, 'potentiel gain= ', 47.58074815877004)
2020-3-1
(u'RB Leipzig - Bayer Leverkusen : A', 0.59365564584732)
(' bet =', 7.477176018390011, 'potentiel gain= ', 41.49832690206456)
2020-2-29
(u'Lazio - Bologne : A', 0.5649816393852231)
(' bet =', 6.546959090078852, 'potentiel gain= ', 42.55523408551254)
2020-3-8
(u'Bayern Munich - Augsbourg : A', 0.43330943584442105)
(' bet =', 2.83490449217984, 'potentiel gain= ', 51.02828085923711